# ROAD COST ANALYSIS 

## PART I - DATA PREPARATION


### Introduction
In this data analysis project we will look Itat the cost of road construction in Poland.  is said that road construction in Poland is much more expensive than in neighboring countries and the quality of the new routes does not meet the requirements of the users.

We will check the list of elements of newly build or rebuilt roads and what road elements contribute the most to the high cost of roads in Poland and what does it look like from the inside.

The analysis will be carried out on the basis of real data for which the names of the roads covered by the analysis have been changed.

The input material are pdf files obtained from a reputable polish construction company, the explanation of which is presented below.

The original data contains the following columns:

* 'Lp.': Ordinal number
* 'CPV': Central Product Classification code
* 'Numer Specyfikacji Technicznej': Technical Specification code
* 'Elementy rozliczeniowe': Billing elements
* 'Jednostka': Measure unit
* 'Ilosc': Quantity
* 'Cena jedn': Unit price
* 'Wartosc calkowita': Total value
* 'Droga': Road number
* 'Rok': Year of construction
* 'Kategoria': Category of construction works

### Import Libraries

In [1]:
import pandas as pd
import numpy as np

from functions.pdf_tools import read_pdfs, clean_pdf, match_category
from functions.data_tools import col_per_km, map_col_type 

#### Read pdf files

In [2]:
road_data = read_pdfs('..\Projekt_Analiza_Danych\DATA\*.pdf')

### Initial data clean

#### Check the basic information of the DataFrame

In [3]:
road_data = clean_pdf(road_data)

In [4]:
road_data.head()

Elementy_rozliczeniowe Jednostka Ilosc  \
3  Koszt dostosowania się do wymagań ogólnych zaw...   ryczałt  1.00   
4  Dokumentacja realizacyjno-technologiczna i dok...   ryczałt  1.00   
5  Koszt utrzymania dróg, po których odbywać się ...   ryczałt  1.00   
6  Saperskie sprawdzenie terenu pod kątem niewypa...   ryczałt  1.00   
7  Nadzór archeologiczny wraz z wykonaniem ratown...   ryczałt  1.00   

   Cena_jedn Wartosc_calkowita   Droga   Rok       Kategoria  
3  280864.20         280764.20  DK12ZW  2019  WARUNKI OGÓLNE  
4   33703.70          33603.70  DK12ZW  2019  WARUNKI OGÓLNE  
5  224691.36         224591.36  DK12ZW  2019  WARUNKI OGÓLNE  
6   33703.70          33603.70  DK12ZW  2019  WARUNKI OGÓLNE  
7   28086.42          27986.42  DK12ZW  2019  WARUNKI OGÓLNE

In [5]:
road_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2105 entries, 3 to 15
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Elementy_rozliczeniowe  2105 non-null   object
 1   Jednostka               2105 non-null   object
 2   Ilosc                   2105 non-null   object
 3   Cena_jedn               2105 non-null   object
 4   Wartosc_calkowita       2105 non-null   object
 5   Droga                   2105 non-null   object
 6   Rok                     2105 non-null   object
 7   Kategoria               2105 non-null   object
dtypes: object(8)
memory usage: 148.0+ KB


In [6]:
road_data.describe()

Elementy_rozliczeniowe Jednostka Ilosc Cena_jedn Wartosc_calkowita  \
count                    2105      2105  2105      2105              2105   
unique                    646        12  1482       769              1967   
top               Tablice "E"        m2  1.00  16851.85          16751.85   
freq                       19       731   130        16                20   

       Droga   Rok              Kategoria  
count   2105  2105                   2105  
unique    19     7                     14  
top     DK77  2021  ROBOTY PRZYGOTOWAWCZE  
freq     156   538                    563

In [7]:
road_data['Kategoria'].value_counts()

ROBOTY PRZYGOTOWAWCZE                                 563
OZNAKOWANIA DRÓG I URZĄDZENIA BEZPIECZEŃSTWA RUCHU    343
PODBUDOWY                                             272
NAWIERZCHNIE                                          205
ZIELEŃ DROGOWA                                        169
ELEMENTY ULIC                                         151
ROBOTY WYKOŃCZENIOWE                                  139
WARUNKI OGÓLNE                                         93
ROBOTY ZIEMNE                                          83
ODWODNIENIE KORPUSU DROGOWEGO                          46
INNE ROBOTY                                            36
WYMAGANIA OGÓLNE                                        3
WARSTWA MROZOOCHRONNA Z MIESZANKI NIEZWIĄZANEJ          1
OZNAKOWANIA DRÓG I URZĄDZENIA                           1
Name: Kategoria, dtype: int64

In [8]:
[ print(f' Column {i} type: {set(road_data[i].map(type))}') for i in road_data.columns]

 Column Elementy_rozliczeniowe type: {<class 'str'>}
 Column Jednostka type: {<class 'str'>}
 Column Ilosc type: {<class 'str'>}
 Column Cena_jedn type: {<class 'str'>}
 Column Wartosc_calkowita type: {<class 'str'>}
 Column Droga type: {<class 'str'>}
 Column Rok type: {<class 'str'>}
 Column Kategoria type: {<class 'str'>}


[None, None, None, None, None, None, None, None]

##### Wrong data types**

As you can see above, the "Ilosc, Cena_jedn, Wartosc_calkowita, Dlugosc_drogi and Rok" columns contains wrong data type, we will convert them to float and int data type. What is more the "Kategoria" column contain incomplete category names. In the next few steps we will fix these errors.

##### Clean the "Kategoria" column

In [9]:
road_data['Kategoria_robot'] = road_data['Kategoria'].apply(match_category)
road_data.drop('Kategoria', inplace=True, axis=1)

In [10]:
road_data.head()

Elementy_rozliczeniowe Jednostka Ilosc  \
3  Koszt dostosowania się do wymagań ogólnych zaw...   ryczałt  1.00   
4  Dokumentacja realizacyjno-technologiczna i dok...   ryczałt  1.00   
5  Koszt utrzymania dróg, po których odbywać się ...   ryczałt  1.00   
6  Saperskie sprawdzenie terenu pod kątem niewypa...   ryczałt  1.00   
7  Nadzór archeologiczny wraz z wykonaniem ratown...   ryczałt  1.00   

   Cena_jedn Wartosc_calkowita   Droga   Rok Kategoria_robot  
3  280864.20         280764.20  DK12ZW  2019  WARUNKI OGOLNE  
4   33703.70          33603.70  DK12ZW  2019  WARUNKI OGOLNE  
5  224691.36         224591.36  DK12ZW  2019  WARUNKI OGOLNE  
6   33703.70          33603.70  DK12ZW  2019  WARUNKI OGOLNE  
7   28086.42          27986.42  DK12ZW  2019  WARUNKI OGOLNE

##### Clean the "Ilosc", "Cena_jedn", "Wartosc_calkowita" acolumns

In [11]:
road_data['Ilosc'] = pd.to_numeric(road_data['Ilosc'],errors='coerce')
road_data['Cena_jedn'] = pd.to_numeric(road_data['Cena_jedn'],errors='coerce')
road_data['Wartosc_calkowita'] = pd.to_numeric(road_data['Wartosc_calkowita'],errors='coerce')
set(road_data['Ilosc'].map(type))

{float}

##### Clean the "Rok" column 

In [12]:
road_data['Rok'] = pd.to_numeric(road_data['Rok'],errors='coerce')
road_data['Rok'] = pd.to_datetime(road_data['Rok'],format='%Y')
road_data['Rok'] = road_data['Rok'].apply(lambda date:date.year)

In [13]:
road_data.head()

Elementy_rozliczeniowe Jednostka  Ilosc  \
3  Koszt dostosowania się do wymagań ogólnych zaw...   ryczałt    1.0   
4  Dokumentacja realizacyjno-technologiczna i dok...   ryczałt    1.0   
5  Koszt utrzymania dróg, po których odbywać się ...   ryczałt    1.0   
6  Saperskie sprawdzenie terenu pod kątem niewypa...   ryczałt    1.0   
7  Nadzór archeologiczny wraz z wykonaniem ratown...   ryczałt    1.0   

   Cena_jedn  Wartosc_calkowita   Droga   Rok Kategoria_robot  
3  280864.20          280764.20  DK12ZW  2019  WARUNKI OGOLNE  
4   33703.70           33603.70  DK12ZW  2019  WARUNKI OGOLNE  
5  224691.36          224591.36  DK12ZW  2019  WARUNKI OGOLNE  
6   33703.70           33603.70  DK12ZW  2019  WARUNKI OGOLNE  
7   28086.42           27986.42  DK12ZW  2019  WARUNKI OGOLNE

In [14]:
road_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2105 entries, 3 to 15
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Elementy_rozliczeniowe  2105 non-null   object 
 1   Jednostka               2105 non-null   object 
 2   Ilosc                   2103 non-null   float64
 3   Cena_jedn               2105 non-null   float64
 4   Wartosc_calkowita       2103 non-null   float64
 5   Droga                   2105 non-null   object 
 6   Rok                     2105 non-null   int64  
 7   Kategoria_robot         2105 non-null   object 
dtypes: float64(3), int64(1), object(4)
memory usage: 148.0+ KB


##### Add new column with road category info

In [15]:
road_data['Kategoria_drogi'] = road_data['Droga'].str[0:2].apply(lambda x: ('Powiatowa' if x=='DP'
                                                     else 'Krajowa' if x=='DK'
                                                     else 'Wojewodzka'))

In [16]:
road_data.head()

Elementy_rozliczeniowe Jednostka  Ilosc  \
3  Koszt dostosowania się do wymagań ogólnych zaw...   ryczałt    1.0   
4  Dokumentacja realizacyjno-technologiczna i dok...   ryczałt    1.0   
5  Koszt utrzymania dróg, po których odbywać się ...   ryczałt    1.0   
6  Saperskie sprawdzenie terenu pod kątem niewypa...   ryczałt    1.0   
7  Nadzór archeologiczny wraz z wykonaniem ratown...   ryczałt    1.0   

   Cena_jedn  Wartosc_calkowita   Droga   Rok Kategoria_robot Kategoria_drogi  
3  280864.20          280764.20  DK12ZW  2019  WARUNKI OGOLNE         Krajowa  
4   33703.70           33603.70  DK12ZW  2019  WARUNKI OGOLNE         Krajowa  
5  224691.36          224591.36  DK12ZW  2019  WARUNKI OGOLNE         Krajowa  
6   33703.70           33603.70  DK12ZW  2019  WARUNKI OGOLNE         Krajowa  
7   28086.42           27986.42  DK12ZW  2019  WARUNKI OGOLNE         Krajowa

In [17]:
road_data['Kategoria_drogi'].value_counts()

Wojewodzka    1307
Krajowa        696
Powiatowa      102
Name: Kategoria_drogi, dtype: int64

##### Road length

We still dont know the lenght of analized roads, so their cost are not reliable. Let's bring all the costs down to the cost of the 1 km of the road.

We know that the cells that contains sentence "Odtworzenie trasy i punktów wysokościowych..." in column "Elementy_rozliczenione" contains information about the lenght of the analized roads. We can use it to bring all the costs down to the cost of the 1 km.

In [18]:
road_elem_df = road_data[road_data['Elementy_rozliczeniowe'].str.contains("Odtworzenie trasy i punktów wysokościowych")]['Droga']
road_quant_df = road_data[road_data['Elementy_rozliczeniowe'].str.contains("Odtworzenie trasy i punktów wysokościowych")]['Ilosc']

In [19]:
road_length_dict = dict(zip(road_elem_df, road_quant_df))
road_length_dict

{'DK12ZW': 24.24,
 'DK33WR': 6.26,
 'DK35WY': 6.34,
 'DK77': 10.0,
 'DK99': 12.88,
 'DP55818': 9.9,
 'DW258': 3.9,
 'DW323R': 12.27,
 'DW534C': 12.82,
 'DW555': 19.47,
 'DW606SLK': 0.8,
 'DW614SK': 16.64,
 'DW659': 7.08,
 'DW741': 10.39,
 'DW763B': 12.46,
 'DW777SX': 18.05,
 'DW789': 2.0,
 'DW878RT': 10.78,
 'DW919SK': 12.98}

In [20]:
road_data['Dlugosc_drogi'] = road_data['Droga'].map(road_length_dict)
road_data

Elementy_rozliczeniowe Jednostka   Ilosc  \
3   Koszt dostosowania się do wymagań ogólnych zaw...   ryczałt    1.00   
4   Dokumentacja realizacyjno-technologiczna i dok...   ryczałt    1.00   
5   Koszt utrzymania dróg, po których odbywać się ...   ryczałt    1.00   
6   Saperskie sprawdzenie terenu pod kątem niewypa...   ryczałt    1.00   
7   Nadzór archeologiczny wraz z wykonaniem ratown...   ryczałt    1.00   
..                                                ...       ...     ...   
6                         Mieszanka roślin rabatowych        m2  100.00   
10  Ścianki oporowe typu "L" - gr 25 cm, wysokość ...         m   51.00   
11  Ścianki oporowe typu "L" - gr. 10 cm, wysokość...         m    8.00   
13  Wykonanie schodów i zejść do posesji z element...        m2   25.16   
15  Pomiar powykonawczy zrealizowanych obiektów dr...        km   12.98   

    Cena_jedn  Wartosc_calkowita    Droga   Rok Kategoria_robot  \
3   280864.20          280764.20   DK12ZW  2019  WARUNKI OGOLNE   
4    33703.70           33603.70   DK12ZW  2019  WARUNKI OGOLNE   
5   224691.36          224591.36   DK12ZW  2019  WARUNKI OGOLNE   
6    33703.70           33603.70   DK12ZW  2019  WARUNKI OGOLNE   
7    28086.42           27986.42   DK12ZW  2019  WARUNKI OGOLNE   
..        ...                ...      ...   ...             ...   
6       50.00            5000.00  DW919SK  2015  ZIELEN DROGOWA   
10    1024.40           52244.40  DW919SK  2015     INNE ROBOTY   
11     274.30            2194.40  DW919SK  2015     INNE ROBOTY   
13     246.34            6198.04  DW919SK  2015     INNE ROBOTY   
15   29829.22          387064.26  DW919SK  2015     INNE ROBOTY   

   Kategoria_drogi  Dlugosc_drogi  
3          Krajowa          24.24  
4          Krajowa          24.24  
5          Krajowa          24.24  
6          Krajowa          24.24  
7          Krajowa          24.24  
..             ...            ...  
6       Wojewodzka          12.98  
10      Wojewodzka          12.98  
11      Wojewodzka          12.98  
13      Wojewodzka          12.98  
15      Wojewodzka          12.98  

[2105 rows x 10 columns]

##### Check the calculations inside the dataframe

It can be seen that the product of the "Ilosc" and "Cena_jedn" columns does not equal to the values in the column "Wartosc_calkowita", let's fix that.

In [21]:
road_data['Wartosc_calkowita'] = road_data['Ilosc'] * road_data['Cena_jedn']

In [22]:
road_data

Elementy_rozliczeniowe Jednostka   Ilosc  \
3   Koszt dostosowania się do wymagań ogólnych zaw...   ryczałt    1.00   
4   Dokumentacja realizacyjno-technologiczna i dok...   ryczałt    1.00   
5   Koszt utrzymania dróg, po których odbywać się ...   ryczałt    1.00   
6   Saperskie sprawdzenie terenu pod kątem niewypa...   ryczałt    1.00   
7   Nadzór archeologiczny wraz z wykonaniem ratown...   ryczałt    1.00   
..                                                ...       ...     ...   
6                         Mieszanka roślin rabatowych        m2  100.00   
10  Ścianki oporowe typu "L" - gr 25 cm, wysokość ...         m   51.00   
11  Ścianki oporowe typu "L" - gr. 10 cm, wysokość...         m    8.00   
13  Wykonanie schodów i zejść do posesji z element...        m2   25.16   
15  Pomiar powykonawczy zrealizowanych obiektów dr...        km   12.98   

    Cena_jedn  Wartosc_calkowita    Droga   Rok Kategoria_robot  \
3   280864.20        280864.2000   DK12ZW  2019  WARUNKI OGOLNE   
4    33703.70         33703.7000   DK12ZW  2019  WARUNKI OGOLNE   
5   224691.36        224691.3600   DK12ZW  2019  WARUNKI OGOLNE   
6    33703.70         33703.7000   DK12ZW  2019  WARUNKI OGOLNE   
7    28086.42         28086.4200   DK12ZW  2019  WARUNKI OGOLNE   
..        ...                ...      ...   ...             ...   
6       50.00          5000.0000  DW919SK  2015  ZIELEN DROGOWA   
10    1024.40         52244.4000  DW919SK  2015     INNE ROBOTY   
11     274.30          2194.4000  DW919SK  2015     INNE ROBOTY   
13     246.34          6197.9144  DW919SK  2015     INNE ROBOTY   
15   29829.22        387183.2756  DW919SK  2015     INNE ROBOTY   

   Kategoria_drogi  Dlugosc_drogi  
3          Krajowa          24.24  
4          Krajowa          24.24  
5          Krajowa          24.24  
6          Krajowa          24.24  
7          Krajowa          24.24  
..             ...            ...  
6       Wojewodzka          12.98  
10      Wojewodzka          12.98  
11      Wojewodzka          12.98  
13      Wojewodzka          12.98  
15      Wojewodzka          12.98  

[2105 rows x 10 columns]

##### New columns with addition factors

Now our dataframe is cleaned we use it to create new columns averaging the cost values depending on road category and road length

In [23]:
road_data = col_per_km(road_data,'Wartosc_calkowita_per_km','Wartosc_calkowita')

In [24]:
cat_elem_df = road_data[road_data['Elementy_rozliczeniowe'].str.contains("Odtworzenie trasy i punktów wysokościowych")]['Kategoria_drogi']
cat_quant_df = road_data[road_data['Elementy_rozliczeniowe'].str.contains("Odtworzenie trasy i punktów wysokościowych")]['Ilosc']
cat_quant_df = pd.concat([cat_elem_df, cat_quant_df], axis=1)
cat_quant_df.head()

Kategoria_drogi  Ilosc
14         Krajowa  24.24
14         Krajowa   6.26
14         Krajowa   6.34
14         Krajowa  10.00
10         Krajowa  12.88

In [25]:
cat_quant_gruper_df = cat_quant_df.groupby(['Kategoria_drogi'], as_index=False).agg({'Ilosc': 'sum'}).sort_values(by=['Ilosc'],ascending=False)

In [26]:
road_cat_dict = dict(cat_quant_gruper_df.values)

In [27]:
road_data['Calk_dl_kat_drogi'] = road_data['Kategoria_drogi'].map(road_cat_dict)
road_data['wsp_kat_drogi'] = road_data['Dlugosc_drogi'] / road_data['Calk_dl_kat_drogi']
road_all_length = sum(road_cat_dict.values())

##### New columns with costs multplied by road category factor

In [28]:
road_data['Wartosc_calkowita_kat'] = road_data['Wartosc_calkowita'] * road_data['wsp_kat_drogi']
road_data['Wartosc_calkowita_per_km_kat'] = road_data['Wartosc_calkowita_per_km'] * road_data['wsp_kat_drogi']

##### New columns with costs multplied by all roads factor

In [29]:
road_data['Dlugosc_wszystkich_drog'] = road_all_length
road_data['wsp_calk_drogi'] = road_data['Dlugosc_drogi'] / road_data['Dlugosc_wszystkich_drog']
road_data['Wartosc_calkowita_calosc'] = road_data['Wartosc_calkowita'] * road_data['wsp_calk_drogi']
road_data['Wartosc_calkowita_per_km_calosc'] = road_data['Wartosc_calkowita_per_km'] * road_data['wsp_calk_drogi']

##### Clean dataframe from work columns with factors

Cleaning data frame from temporary columns with factors and work lengths

In [30]:
road_data.drop(['wsp_calk_drogi','wsp_kat_drogi','Dlugosc_wszystkich_drog','Calk_dl_kat_drogi'], inplace=True, axis=1)
road_data.head()

Elementy_rozliczeniowe Jednostka  Ilosc  \
3  Koszt dostosowania się do wymagań ogólnych zaw...   ryczałt    1.0   
4  Dokumentacja realizacyjno-technologiczna i dok...   ryczałt    1.0   
5  Koszt utrzymania dróg, po których odbywać się ...   ryczałt    1.0   
6  Saperskie sprawdzenie terenu pod kątem niewypa...   ryczałt    1.0   
7  Nadzór archeologiczny wraz z wykonaniem ratown...   ryczałt    1.0   

   Cena_jedn  Wartosc_calkowita   Droga   Rok Kategoria_robot Kategoria_drogi  \
3  280864.20          280864.20  DK12ZW  2019  WARUNKI OGOLNE         Krajowa   
4   33703.70           33703.70  DK12ZW  2019  WARUNKI OGOLNE         Krajowa   
5  224691.36          224691.36  DK12ZW  2019  WARUNKI OGOLNE         Krajowa   
6   33703.70           33703.70  DK12ZW  2019  WARUNKI OGOLNE         Krajowa   
7   28086.42           28086.42  DK12ZW  2019  WARUNKI OGOLNE         Krajowa   

   Dlugosc_drogi  Wartosc_calkowita_per_km  Wartosc_calkowita_kat  \
3          24.24              11586.806931          114001.142130   
4          24.24               1390.416667           13680.135432   
5          24.24               9269.445545           91200.913704   
6          24.24               1390.416667           13680.135432   
7          24.24               1158.680693           11400.114213   

   Wartosc_calkowita_per_km_kat  Wartosc_calkowita_calosc  \
3                   4703.017415              32534.398394   
4                    564.362023               3904.127344   
5                   3762.413932              26027.518715   
6                    564.362023               3904.127344   
7                    470.301741               3253.439839   

   Wartosc_calkowita_per_km_calosc  
3                      1342.178152  
4                       161.061359  
5                      1073.742521  
6                       161.061359  
7                       134.217815

##### Note

Okey we finished our data preparation let's save it to excel file for further data analysis in second part of the project.

In [31]:
road_data.to_excel('..\Projekt_Analiza_Danych\DATA\Road_cost_analysis.xlsx',
                                             sheet_name='Road_cost_analysis',
                                             index=False)